In [28]:
import pandas as pd
import numpy as np

from datetime import timedelta
from datetime import datetime


In [29]:
data = pd.read_csv('data/2022.csv')

In [3]:
data = data[data['Actividad_Realizada'] == 'Exploración']
data['Resultado'].value_counts()

Resultado
Negativo      9393
Positivo       891
En proceso       9
Name: count, dtype: int64

In [30]:
data.drop(data[data['Resultado']=='En proceso'].index, inplace=True)

In [13]:
data_output = data[['Fecha','Latitud','Longitud','Resultado']]
data_output['Fecha'] = data_output['Fecha'].astype('datetime64[s]')
data_output['Latitude'] = data_output['Latitud']
data_output['Longitude'] = data_output['Longitud']

data_output['Fecha_inicio'] = data_output['Fecha'] - timedelta(days=96)

data_output

/var/folders/9n/kxcs2s054c32h_5hwhhg4bd40000gn/T/ipykernel_35072/2845424671.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output['Fecha'] = data_output['Fecha'].astype('datetime64[s]')
/var/folders/9n/kxcs2s054c32h_5hwhhg4bd40000gn/T/ipykernel_35072/2845424671.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output['Latitude'] = data_output['Latitud']
/var/folders/9n/kxcs2s054c32h_5hwhhg4bd40000gn/T/ipykernel_35072/2845424671.py:4: SettingWithCopyWarning: 
A value is trying to be set on a

,Fecha,Latitud,Longitud,Resultado,Latitude,Longitude,Fecha_inicio
0,2022-07-29,20.54868,-89.93041,Negativo,20.54868,-89.93041,2022-04-24
1,2022-10-31,20.54868,-89.93041,Negativo,20.54868,-89.93041,2022-07-27
2,2022-06-28,20.54868,-89.93041,Negativo,20.54868,-89.93041,2022-03-24
3,2022-06-14,20.54868,-89.93041,Negativo,20.54868,-89.93041,2022-03-10
4,2022-01-31,20.54868,-89.93041,Negativo,20.54868,-89.93041,2021-10-27
...,...,...,...,...,...,...,...
11171,2022-11-10,21.14989,-89.24844,Negativo,21.14989,-89.24844,2022-08-06
11172,2022-11-10,21.15384,-89.25343,Negativo,21.15384,-89.25343,2022-08-06
11173,2022-11-10,21.15146,-89.23945,Negativo,21.15146,-89.23945,2022-08-06
11174,2022-11-10,21.15626,-89.24965,Negativo,21.15626,-89.24965,2022-08-06


In [62]:
df = pd.read_csv('data/ndvi/ndvi_-87.783_21.02725.csv')
df['date'] = pd.to_datetime(df['date'],format='%Y%m%d')
out = df.set_index('date').asfreq('D', method='ffill').reset_index()
out.to_csv('x.csv')

In [23]:
#idx = pd.date_range('2021-09-20', '2021-09-20')

def get_series(row,field_name,name,prefix):
    df = pd.read_csv('nasa-coordinates/'+field_name+'/' + prefix + str(row['Latitude']) + '_' + str(row['Longitude']) + '.csv')
    df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')  
    #df = df.set_index('date').asfreq('D', method='ffill').reset_index()
    mask = (df['Date'] > row['Fecha_inicio']) & (df['Date'] <= row['Fecha'])
    df = df.loc[mask]
    list = df[name].tolist()  
    if   
    return list

#get_series(data_output.iloc[0])['NDVI'].tolist()



In [24]:
#for index, row in data_output.iterrows():
#    print(get_series(row))

data_output['MCD15A3H_061_Lai_500m'] = data_output.apply(lambda row: get_series(row,'MCD15A3H-061','MCD15A3H_061_Lai_500m',''), axis=1)
data_output['MCD15A3H_061_FparExtra_QC'] = data_output.apply(lambda row: get_series(row,'MCD15A3H-061','MCD15A3H_061_FparExtra_QC',''), axis=1)
data_output['MOD11A1_061_LST_Day_1km'] = data_output.apply(lambda row: get_series(row,'MOD11A1-061','MOD11A1_061_LST_Day_1km',''), axis=1)
data_output['MOD11A1_061_LST_Night_1km'] = data_output.apply(lambda row: get_series(row,'MOD11A1-061','MOD11A1_061_LST_Night_1km',''), axis=1)


data_output['MOD13Q1_061__250m_16_days_NDVI'] = data_output.apply(lambda row: get_series(row,'MOD13Q1-061','MOD13Q1_061__250m_16_days_NDVI',''), axis=1)
data_output['MCD15A3H_061_FparExtra_QC'] = data_output.apply(lambda row: get_series(row,'SPL4SMGP-006','SPL4SMGP_006_Geophysical_Data_sm_rootzone_0',''), axis=1)


data_output

/var/folders/9n/kxcs2s054c32h_5hwhhg4bd40000gn/T/ipykernel_35072/3679016688.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output['MCD15A3H_061_Lai_500m'] = data_output.apply(lambda row: get_series(row,'MCD15A3H-061','MCD15A3H_061_Lai_500m',''), axis=1)
/var/folders/9n/kxcs2s054c32h_5hwhhg4bd40000gn/T/ipykernel_35072/3679016688.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output['MCD15A3H_061_FparExtra_QC'] = data_output.apply(lambda row: get_series(row,'MCD15A3H-061','MCD15A3H_061_Fpa

,Fecha,Latitud,Longitud,Resultado,Latitude,Longitude,Fecha_inicio,MCD15A3H_061_Lai_500m,MCD15A3H_061_FparExtra_QC,MOD11A1_061_LST_Day_1km,MOD13Q1_061__250m_16_days_NDVI
0,2022-07-29,20.54868,-89.93041,Negativo,20.54868,-89.93041,2022-04-24,"[0.3, 0.3, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, ...","[0.1842769, 0.1841989, 0.1862786, 0.1988801, 0...","[312.3200073, 0.0, 307.1199951, 0.0, 304.38000...","[0.5961, 0.5961, 0.5961, 0.5961, 0.5961, 0.596..."
1,2022-10-31,20.54868,-89.93041,Negativo,20.54868,-89.93041,2022-07-27,"[1.7, 1.7, 1.7, 1.7, 1.9, 1.9, 1.9, 1.9, 1.5, ...","[0.2353308, 0.2292294, 0.2403743, 0.2456544, 0...","[0.0, 0.0, 0.0, 299.1799927, 303.3800049, 300....","[0.7487, 0.7487, 0.7487, 0.7487, 0.7487, 0.748..."
2,2022-06-28,20.54868,-89.93041,Negativo,20.54868,-89.93041,2022-03-24,"[0.6, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, ...","[0.209803, 0.2055995, 0.2032753, 0.2011252, 0....","[302.4799805, 304.3200073, 305.4799805, 0.0, 3...","[0.5306, 0.5306, 0.5306, 0.5306, 0.5306, 0.530..."
3,2022-06-14,20.54868,-89.93041,Negativo,20.54868,-89.93041,2022-03-10,"[1.0, 1.0, 1.0, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, ...","[0.1884649, 0.1875311, 0.1880844, 0.1878469, 0...","[307.2799988, 0.0, 0.0, 302.1199951, 303.89999...","[0.6227, 0.6227, 0.6227, 0.6227, 0.6227, 0.622..."
4,2022-01-31,20.54868,-89.93041,Negativo,20.54868,-89.93041,2021-10-27,"[2.0, 2.0, 2.0, 2.0, 1.9, 1.9, 1.9, 1.9, 1.2, ...","[0.2188734, 0.219951, 0.2171111, 0.2155131, 0....","[300.1199951, 300.1999817, 0.0, 301.7399902, 3...","[0.8415, 0.8415, 0.8415, 0.8415, 0.7837, 0.783..."
...,...,...,...,...,...,...,...,...,...,...,...
11171,2022-11-10,21.14989,-89.24844,Negativo,21.14989,-89.24844,2022-08-06,"[2.5, 2.5, 2.2, 2.2, 2.2, 2.2, 1.7, 1.7, 1.7, ...","[0.2431191, 0.2520516, 0.2624414, 0.2578658, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 301.7200012, 0....","[0.7389, 0.7389, 0.7389, 0.7389, 0.7389, 0.738..."
11172,2022-11-10,21.15384,-89.25343,Negativo,21.15384,-89.25343,2022-08-06,"[2.4000001, 2.4000001, 2.4000001, 2.4000001, 2...","[0.2468423, 0.2547372, 0.2663476, 0.2606734, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 301.9400024, 0....","[0.7896, 0.7896, 0.7896, 0.7896, 0.7896, 0.789..."
11173,2022-11-10,21.15146,-89.23945,Negativo,21.15146,-89.23945,2022-08-06,"[2.7, 2.7, 2.6000001, 2.6000001, 2.6000001, 2....","[0.2431191, 0.2520516, 0.2624414, 0.2578658, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 301.5799866, 0....","[0.7819, 0.7819, 0.7819, 0.7819, 0.7819, 0.781..."
11174,2022-11-10,21.15626,-89.24965,Negativo,21.15626,-89.24965,2022-08-06,"[6.8000002, 6.8000002, 5.5, 5.5, 5.5, 5.5, 1.8...","[0.2444008, 0.2525399, 0.2651269, 0.2604293, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 301.9400024, 0....","[0.7993, 0.7993, 0.7993, 0.7993, 0.7993, 0.799..."


In [32]:

dataset = data_output[['MCD15A3H_061_Lai_500m','MCD15A3H_061_FparExtra_QC','MOD11A1_061_LST_Day_1km','MOD13Q1_061__250m_16_days_NDVI','Resultado']]

dataset.to_csv('2022-dataset.csv', index=False)